In [1]:
import pandas as pd
import numpy as np

from tqdm import tqdm_notebook

DATA = 'data/'

In [2]:
train_df = pd.read_hdf(DATA + 'train_augmented.h5', key='train')
items_df = pd.read_csv(DATA + 'items.csv')
item_categories_df = pd.read_csv(DATA + 'item_categories.csv')
shops_df = pd.read_csv(DATA + 'shops.csv')

train_df['item_id'] = train_df.item_id.astype(np.int32)
train_df['shop_id'] = train_df.shop_id.astype(np.int32)

In [3]:
train_df.head()
items_df.head()

,date_block_num,item_price,item_cnt_day,shop_id,item_id
0,0,1999.0,0.0,5,5037
1,0,999.0,0.0,50,19034
2,0,799.0,0.0,35,15451
3,0,149.0,0.0,50,18968
4,0,299.0,0.0,50,18047


,item_name,item_id,item_category_id
0,! ВО ВЛАСТИ НАВАЖДЕНИЯ (ПЛАСТ.) D,0,40
1,!ABBYY FineReader 12 Professional Edition Full...,1,76
2,***В ЛУЧАХ СЛАВЫ (UNV) D,2,40
3,***ГОЛУБАЯ ВОЛНА (Univ) D,3,40
4,***КОРОБКА (СТЕКЛО) D,4,40


In [4]:
train_df[['date_block_num', 'item_id']].nunique()

date_block_num      34
item_id           5100
dtype: int64

In [9]:
train_df.dtypes

date_block_num      int64
item_price        float64
item_cnt_day      float64
shop_id            object
item_id            object
dtype: object

In [11]:
train_df = train_df.merge(items_df, how='left', on='item_id')
train_df = train_df.merge(shops_df, on='shop_id')

train_df.head()

,date_block_num,item_price,item_cnt_day,shop_id,item_id,item_name,item_category_id,shop_name,city_id
0,0,1999.0,0.0,5,5037,"NHL 15 [PS3, русские субтитры]",19,"Вологда ТРЦ ""Мармелад""",5
1,0,1299.0,0.0,5,1559,Assassin's Creed: Единство. Стандартное издани...,31,"Вологда ТРЦ ""Мармелад""",5
2,0,499.0,0.0,5,1519,"Assassin's Creed Liberation HD [PC, Цифровая в...",31,"Вологда ТРЦ ""Мармелад""",5
3,0,1999.0,0.0,5,1435,"Anno 2205. Standard Edition [PC, Цифровая версия]",31,"Вологда ТРЦ ""Мармелад""",5
4,0,749.0,0.0,5,1562,"Assassin's Creed: Изгой [PC, Цифровая версия]",31,"Вологда ТРЦ ""Мармелад""",5


#### Sums by month shop and item

In [12]:
sums = train_df.groupby(by=['date_block_num', 'shop_id', 'item_id'])['item_cnt_day'].sum().reset_index()
sums.to_csv(DATA + 'month_item_shop_sums.csv', index=False)

#### Sums by month shop and item category

In [13]:
sums = train_df.groupby(by=['date_block_num', 'shop_id', 'item_category_id'])['item_cnt_day'].sum().reset_index()
sums.to_csv(DATA + 'month_item_category_shop_sums.csv', index=False)

#### Sums by month city and item

In [15]:
sums = train_df.groupby(by=['date_block_num', 'city_id', 'item_id'])['item_cnt_day'].sum().reset_index()
sums.to_csv(DATA + 'month_item_city_sums.csv', index=False)

#### Sums by month city and item category

In [16]:
sums = train_df.groupby(by=['date_block_num', 'city_id', 'item_category_id'])['item_cnt_day'].sum().reset_index()
sums.to_csv(DATA + 'month_item_category_city_sums.csv', index=False)

#### Cumsum and comcount for item and shop

In [17]:
grouped = train_df.groupby(by=['item_id', 'shop_id'])
item_shop_aggregated_df = grouped['item_cnt_day'].agg(['cumsum', 'cumcount'])
item_shop_aggregated_df.to_csv(DATA + 'item_shop_cumsum_cumcount.csv', index=False)

#### Cumsum and comcount for item category and shop

In [18]:
grouped = train_df.groupby(by=['item_category_id', 'shop_id'])
item_category_shop_aggregated_df = grouped['item_cnt_day'].agg(['cumsum', 'cumcount'])
item_category_shop_aggregated_df.to_csv(DATA + 'item_category_shop_cumsum_cumcount.csv', index=False)

#### Cumsum and comcount for item and city

In [19]:
grouped = train_df.groupby(by=['item_id', 'city_id'])
item_city_aggregated_df = grouped['item_cnt_day'].agg(['cumsum', 'cumcount'])
item_city_aggregated_df.to_csv(DATA + 'item_city_cumsum_cumcount.csv', index=False)

#### Cumsum and comcount for item category and city

In [20]:
grouped = train_df.groupby(by=['item_category_id', 'city_id'])
item_category_city_aggregated_df = grouped['item_cnt_day'].agg(['cumsum', 'cumcount'])
item_category_city_aggregated_df.to_csv(DATA + 'item_category_city_cumsum_cumcount.csv', index=False)

#### Last prices

In [21]:
tmp = train_df[['item_id', 'item_price']]

tmp = tmp[tmp.item_price > 0]
tmp.drop_duplicates(keep='last', inplace=True)

tmp.head(15)

,item_id,item_price
7654,8094,1010.120
9091,10230,1598.800
10109,3719,598.700
10529,5272,1199.000
12402,3328,1299.500
13104,33,399.000
15833,3851,898.700
25864,13901,298.900
26115,15255,1198.900
30668,491,479.700


In [22]:
# changes_df = tmp.groupby(by='item_id').item_price.diff()
# changes_df[changes_df.notna()] = 1
# changes_df.fillna(0, inplace=True)
# changes_df = changes_df.to_frame(name='diff')
# changes_df.to_csv(DATA + 'changes.csv')

In [23]:
last_prices_df = tmp.drop_duplicates(subset=['item_id'], keep='last')
last_prices_df.to_csv(DATA + 'last_prices.csv')

### Feature merge

In [24]:
feature_df = train_df.copy()

In [6]:
TIME_LAG = [1, 3, 6, 9, 12]

In [26]:
for i in TIME_LAG:
    feature_df['lag%s' % i] = feature_df['date_block_num'] - i

In [34]:
feature_df.drop('shop_name', inplace=True, axis=1)

feature_df.drop('item_name', inplace=True, axis=1)

#### Item month shop sums

In [27]:
df = pd.read_csv(DATA + 'month_item_shop_sums.csv')

In [28]:
for i in TIME_LAG:
    left = ['lag%s' % i, 'item_id', 'shop_id']
    right = ['date_block_num', 'item_id', 'shop_id']
    feature_df = feature_df.merge(df, how='left', left_on=left, right_on=right, suffixes=['', '_%s' % i])

In [35]:
feature_df.shape
train_df.shape

feature_df.head()

(7282800, 17)

(7282800, 9)

,date_block_num,item_price,item_cnt_day,shop_id,item_id,item_category_id,city_id,lag1,lag3,lag6,lag9,lag12,item_lag_1,item_lag_3,item_lag_6,item_lag_9,item_lag_12
0,0,1999.0,0.0,5,5037,19,5,-1,-3,-6,-9,-12,0.0,0.0,0.0,0.0,0.0
1,0,1299.0,0.0,5,1559,31,5,-1,-3,-6,-9,-12,0.0,0.0,0.0,0.0,0.0
2,0,499.0,0.0,5,1519,31,5,-1,-3,-6,-9,-12,0.0,0.0,0.0,0.0,0.0
3,0,1999.0,0.0,5,1435,31,5,-1,-3,-6,-9,-12,0.0,0.0,0.0,0.0,0.0
4,0,749.0,0.0,5,1562,31,5,-1,-3,-6,-9,-12,0.0,0.0,0.0,0.0,0.0


In [31]:
feature_df.drop(['date_block_num_%s' % i for i in TIME_LAG], inplace=True, axis=1)

feature_df.rename(columns={"item_cnt_day_%s" %i : "item_lag_%s" % i for i in TIME_LAG}, inplace=True)

for i in TIME_LAG:
    feature_df['item_lag_%s' % i].fillna(0, inplace=True)

In [36]:
feature_df.to_csv(DATA + 'features.csv', index=False)

#### Item category month shop sums

In [37]:
df = pd.read_csv(DATA + 'month_item_category_shop_sums.csv')

In [38]:
for i in TIME_LAG:
    left = ['lag%s' % i, 'item_category_id', 'shop_id']
    right = ['date_block_num', 'item_category_id', 'shop_id']
    feature_df = feature_df.merge(df, how='left', left_on=left, right_on=right, suffixes=['', '_%s' % i])

In [42]:
feature_df.shape
train_df.shape

df.head()
feature_df.head()

(7282800, 22)

(7282800, 9)

,date_block_num,shop_id,item_category_id,item_cnt_day
0,0,2,0,0.0
1,0,2,2,10.0
2,0,2,3,0.0
3,0,2,5,2.0
4,0,2,6,10.0


,date_block_num,item_price,item_cnt_day,shop_id,item_id,item_category_id,city_id,lag1,lag3,lag6,...,item_lag_1,item_lag_3,item_lag_6,item_lag_9,item_lag_12,item_category_lag_1,item_category_lag_3,item_category_lag_6,item_category_lag_9,item_category_lag_12
0,0,1999.0,0.0,5,5037,19,5,-1,-3,-6,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0,1299.0,0.0,5,1559,31,5,-1,-3,-6,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0,499.0,0.0,5,1519,31,5,-1,-3,-6,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0,1999.0,0.0,5,1435,31,5,-1,-3,-6,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0,749.0,0.0,5,1562,31,5,-1,-3,-6,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [40]:
feature_df.drop(['date_block_num_%s' % i for i in TIME_LAG], inplace=True, axis=1)

feature_df.rename(columns={"item_cnt_day_%s" %i : "item_category_lag_%s" % i for i in TIME_LAG}, inplace=True)

for i in TIME_LAG:
    feature_df['item_category_lag_%s' % i].fillna(0, inplace=True)

In [41]:
feature_df.shape
train_df.shape

(7282800, 22)

(7282800, 9)

In [106]:
# feature_df.to_csv(DATA + 'features.csv', index=False)

#### Item shop cumsums

In [43]:
df = pd.read_csv(DATA + 'item_shop_cumsum_cumcount.csv')
df.shape
df.head()

(7282800, 2)

,cumsum,cumcount
0,0.0,0
1,0.0,0
2,0.0,0
3,0.0,0
4,0.0,0


In [44]:
tmp = feature_df.join(item_shop_aggregated_df)
tmp.rename(columns={'cumsum': 'item_cumsum', 'cumcount': 'item_cumcount'}, inplace=True)
tmp.item_cumsum = tmp.item_cumsum - tmp.item_cnt_day

In [48]:
tmp.shape
tmp.head()

(7282800, 25)

,date_block_num,item_price,item_cnt_day,shop_id,item_id,item_category_id,city_id,lag1,lag3,lag6,...,item_lag_9,item_lag_12,item_category_lag_1,item_category_lag_3,item_category_lag_6,item_category_lag_9,item_category_lag_12,item_cumsum,item_cumcount,item_avg
0,0,1999.0,0.0,5,5037,19,5,-1,-3,-6,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0
1,0,1299.0,0.0,5,1559,31,5,-1,-3,-6,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0
2,0,499.0,0.0,5,1519,31,5,-1,-3,-6,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0
3,0,1999.0,0.0,5,1435,31,5,-1,-3,-6,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0
4,0,749.0,0.0,5,1562,31,5,-1,-3,-6,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0


In [47]:
mask = tmp.item_cumcount != 0
tmp['item_avg'] = 0
tmp.loc[mask, 'item_avg'] = tmp[mask].item_cumsum / tmp[mask].item_cumcount

In [50]:
feature_df = tmp.copy()
# feature_df.to_csv(DATA + 'features.csv', index=False)

In [51]:
df =pd.read_csv(DATA + 'item_category_shop_cumsum_cumcount.csv')
df.shape
df.head()

(7282800, 2)

,cumsum,cumcount
0,0.0,0
1,0.0,0
2,0.0,1
3,0.0,2
4,0.0,3


In [52]:
tmp = feature_df.join(item_category_shop_aggregated_df)
tmp.rename(columns={'cumsum': 'item_category_cumsum', 'cumcount': 'item_category_cumcount'}, inplace=True)
tmp.item_category_cumsum = tmp.item_category_cumsum - tmp.item_cnt_day

In [53]:
tmp.shape
tmp.head()

(7282800, 27)

,date_block_num,item_price,item_cnt_day,shop_id,item_id,item_category_id,city_id,lag1,lag3,lag6,...,item_category_lag_1,item_category_lag_3,item_category_lag_6,item_category_lag_9,item_category_lag_12,item_cumsum,item_cumcount,item_avg,item_category_cumsum,item_category_cumcount
0,0,1999.0,0.0,5,5037,19,5,-1,-3,-6,...,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0,0
1,0,1299.0,0.0,5,1559,31,5,-1,-3,-6,...,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0,0
2,0,499.0,0.0,5,1519,31,5,-1,-3,-6,...,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0,1
3,0,1999.0,0.0,5,1435,31,5,-1,-3,-6,...,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0,2
4,0,749.0,0.0,5,1562,31,5,-1,-3,-6,...,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0,3


In [54]:
mask = tmp.item_category_cumcount != 0
tmp['item_category_avg'] = 0
tmp.loc[mask, 'item_category_avg'] = tmp[mask].item_category_cumsum / tmp[mask].item_category_cumcount

In [55]:
feature_df = tmp.copy()
feature_df.to_csv(DATA + 'features.csv', index=False)

In [56]:
df = pd.read_csv(DATA + 'month_item_city_sums.csv')

df.shape
df.head()

(4681800, 4)

,date_block_num,city_id,item_id,item_cnt_day
0,0,1,30,0.0
1,0,1,31,0.0
2,0,1,32,13.0
3,0,1,33,3.0
4,0,1,38,0.0


In [58]:
tmp = feature_df.copy()

for i in TIME_LAG:
    left = ['lag%s' % i, 'item_id', 'city_id']
    right = ['date_block_num', 'item_id', 'city_id']
    tmp = tmp.merge(df, how='left', left_on=left, right_on=right, suffixes=['', '_%s' % i])

In [61]:
tmp.shape
train_df.shape

tmp.head()

(7282800, 33)

(7282800, 9)

,date_block_num,item_price,item_cnt_day,shop_id,item_id,item_category_id,city_id,lag1,lag3,lag6,...,item_cumcount,item_avg,item_category_cumsum,item_category_cumcount,item_category_avg,item_city_lag_1,item_city_lag_3,item_city_lag_6,item_city_lag_9,item_city_lag_12
0,0,1999.0,0.0,5,5037,19,5,-1,-3,-6,...,0,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0
1,0,1299.0,0.0,5,1559,31,5,-1,-3,-6,...,0,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0
2,0,499.0,0.0,5,1519,31,5,-1,-3,-6,...,0,0.0,0.0,1,0.0,0.0,0.0,0.0,0.0,0.0
3,0,1999.0,0.0,5,1435,31,5,-1,-3,-6,...,0,0.0,0.0,2,0.0,0.0,0.0,0.0,0.0,0.0
4,0,749.0,0.0,5,1562,31,5,-1,-3,-6,...,0,0.0,0.0,3,0.0,0.0,0.0,0.0,0.0,0.0


In [60]:
tmp.drop(['date_block_num_%s' % i for i in TIME_LAG], inplace=True, axis=1)

tmp.rename(columns={"item_cnt_day_%s" %i : "item_city_lag_%s" % i for i in TIME_LAG}, inplace=True)

for i in TIME_LAG:
    tmp['item_city_lag_%s' % i].fillna(0, inplace=True)

In [62]:
feature_df = tmp.copy()
# feature_df.to_csv(DATA + 'features.csv', index=False)

In [63]:
df = pd.read_csv(DATA + 'month_item_category_city_sums.csv')

df.shape
df.head()

(56916, 4)

,date_block_num,city_id,item_category_id,item_cnt_day
0,0,1,0,0.0
1,0,1,2,32.0
2,0,1,3,0.0
3,0,1,5,7.0
4,0,1,6,67.0


In [64]:
tmp = feature_df.copy()

for i in TIME_LAG:
    left = ['lag%s' % i, 'item_category_id', 'city_id']
    right = ['date_block_num', 'item_category_id', 'city_id']
    tmp = tmp.merge(df, how='left', left_on=left, right_on=right, suffixes=['', '_%s' % i])

In [67]:
tmp.shape
train_df.shape

tmp.head()

(7282800, 38)

(7282800, 9)

,date_block_num,item_price,item_cnt_day,shop_id,item_id,item_category_id,city_id,lag1,lag3,lag6,...,item_city_lag_1,item_city_lag_3,item_city_lag_6,item_city_lag_9,item_city_lag_12,item_category_city_lag_1,item_category_city_lag_3,item_category_city_lag_6,item_category_city_lag_9,item_category_city_lag_12
0,0,1999.0,0.0,5,5037,19,5,-1,-3,-6,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0,1299.0,0.0,5,1559,31,5,-1,-3,-6,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0,499.0,0.0,5,1519,31,5,-1,-3,-6,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0,1999.0,0.0,5,1435,31,5,-1,-3,-6,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0,749.0,0.0,5,1562,31,5,-1,-3,-6,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [66]:
tmp.drop(['date_block_num_%s' % i for i in TIME_LAG], inplace=True, axis=1)

tmp.rename(columns={"item_cnt_day_%s" %i : "item_category_city_lag_%s" % i for i in TIME_LAG}, inplace=True)

for i in TIME_LAG:
    tmp['item_category_city_lag_%s' % i].fillna(0, inplace=True)

In [68]:
feature_df = tmp.copy()
# feature_df.to_csv(DATA + 'features.csv', index=False)

In [69]:
df = pd.read_csv(DATA + 'item_city_cumsum_cumcount.csv')
df.shape
df.head()

(7282800, 2)

,cumsum,cumcount
0,0.0,0
1,0.0,0
2,0.0,0
3,0.0,0
4,0.0,0


In [ ]:
tmp = feature_df.join(item_city_aggregated_df)
tmp.rename(columns={'cumsum': 'item_city_cumsum', 'cumcount': 'item_city_cumcount'}, inplace=True)
tmp.item_city_cumsum = tmp.item_city_cumsum - tmp.item_cnt_day

In [ ]:
tmp.shape
tmp.head()

In [ ]:
mask = tmp.item_city_cumcount != 0
tmp['item_city_avg'] = 0
tmp.loc[mask, 'item_city_avg'] = tmp[mask].item_city_cumsum / tmp[mask].item_city_cumcount

In [153]:
feature_df = tmp.copy()
# feature_df.to_csv(DATA + 'features.csv', index=False)

In [2]:
feature_df = pd.read_csv(DATA + 'features.csv')

In [4]:
feature_df.shape
feature_df.head()

(7282800, 41)

,date_block_num,item_price,item_cnt_day,shop_id,item_id,item_category_id,city_id,lag1,lag3,lag6,...,item_city_lag_9,item_city_lag_12,item_category_city_lag_1,item_category_city_lag_3,item_category_city_lag_6,item_category_city_lag_9,item_category_city_lag_12,item_city_cumsum,item_city_cumcount,item_city_avg
0,0,1999.0,0.0,5,5037,19,5,-1,-3,-6,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0
1,0,1299.0,0.0,5,1559,31,5,-1,-3,-6,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0
2,0,499.0,0.0,5,1519,31,5,-1,-3,-6,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0
3,0,1999.0,0.0,5,1435,31,5,-1,-3,-6,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0
4,0,749.0,0.0,5,1562,31,5,-1,-3,-6,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0


In [5]:
df = pd.read_csv(DATA + 'item_category_city_cumsum_cumcount.csv')
df.shape
df.head()

(7282800, 2)

,cumsum,cumcount
0,0.0,0
1,0.0,0
2,0.0,1
3,0.0,2
4,0.0,3


In [7]:
tmp = feature_df.join(df)
tmp.rename(columns={'cumsum': 'item_category_city_cumsum', 'cumcount': 'item_category_city_cumcount'}, inplace=True)
tmp.item_category_city_cumsum = tmp.item_category_city_cumsum - tmp.item_cnt_day

In [10]:
tmp.shape
tmp.head()

(7282800, 44)

,date_block_num,item_price,item_cnt_day,shop_id,item_id,item_category_id,city_id,lag1,lag3,lag6,...,item_category_city_lag_3,item_category_city_lag_6,item_category_city_lag_9,item_category_city_lag_12,item_city_cumsum,item_city_cumcount,item_city_avg,item_category_city_cumsum,item_category_city_cumcount,item_category_city_avg
0,0,1999.0,0.0,5,5037,19,5,-1,-3,-6,...,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0,0,0.0
1,0,1299.0,0.0,5,1559,31,5,-1,-3,-6,...,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0,0,0.0
2,0,499.0,0.0,5,1519,31,5,-1,-3,-6,...,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0,1,0.0
3,0,1999.0,0.0,5,1435,31,5,-1,-3,-6,...,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0,2,0.0
4,0,749.0,0.0,5,1562,31,5,-1,-3,-6,...,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0,3,0.0


In [9]:
mask = tmp.item_category_city_cumcount != 0
tmp['item_category_city_avg'] = 0
tmp.loc[mask, 'item_category_city_avg'] = tmp[mask].item_category_city_cumsum / tmp[mask].item_category_city_cumcount

In [11]:
feature_df = tmp
feature_df.to_csv(DATA + 'features.csv', index=False)

In [13]:
feature_df.columns

Index(['date_block_num', 'item_price', 'item_cnt_day', 'shop_id', 'item_id',
       'item_category_id', 'city_id', 'lag1', 'lag3', 'lag6', 'lag9', 'lag12',
       'item_lag_1', 'item_lag_3', 'item_lag_6', 'item_lag_9', 'item_lag_12',
       'item_category_lag_1', 'item_category_lag_3', 'item_category_lag_6',
       'item_category_lag_9', 'item_category_lag_12', 'item_cumsum',
       'item_cumcount', 'item_avg', 'item_category_cumsum',
       'item_category_cumcount', 'item_category_avg', 'item_city_lag_1',
       'item_city_lag_3', 'item_city_lag_6', 'item_city_lag_9',
       'item_city_lag_12', 'item_category_city_lag_1',
       'item_category_city_lag_3', 'item_category_city_lag_6',
       'item_category_city_lag_9', 'item_category_city_lag_12',
       'item_city_cumsum', 'item_city_cumcount', 'item_city_avg',
       'item_category_city_cumsum', 'item_category_city_cumcount',
       'item_category_city_avg'],
      dtype='object')

In [16]:
feature_df.drop(['lag%s' % i for i in TIME_LAG], inplace=True, axis=1)

In [17]:
feature_df.shape
feature_df.head()

(7282800, 39)

,date_block_num,item_price,item_cnt_day,shop_id,item_id,item_category_id,city_id,item_lag_1,item_lag_3,item_lag_6,...,item_category_city_lag_3,item_category_city_lag_6,item_category_city_lag_9,item_category_city_lag_12,item_city_cumsum,item_city_cumcount,item_city_avg,item_category_city_cumsum,item_category_city_cumcount,item_category_city_avg
0,0,1999.0,0.0,5,5037,19,5,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0,0,0.0
1,0,1299.0,0.0,5,1559,31,5,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0,0,0.0
2,0,499.0,0.0,5,1519,31,5,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0,1,0.0
3,0,1999.0,0.0,5,1435,31,5,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0,2,0.0
4,0,749.0,0.0,5,1562,31,5,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0,3,0.0


In [18]:
feature_df = feature_df[feature_df.date_block_num > 11]

In [19]:
feature_df.shape
feature_df.head()

(4712400, 39)

,date_block_num,item_price,item_cnt_day,shop_id,item_id,item_category_id,city_id,item_lag_1,item_lag_3,item_lag_6,...,item_category_city_lag_3,item_category_city_lag_6,item_category_city_lag_9,item_category_city_lag_12,item_city_cumsum,item_city_cumcount,item_city_avg,item_category_city_cumsum,item_category_city_cumcount,item_category_city_avg
61200,12,6999.0,0.0,5,3839,24,5,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,12,0.000000,0.0,1140,0.000000
61201,12,899.0,0.0,5,6415,23,5,0.0,1.0,0.0,...,46.0,8.0,10.0,0.0,1.0,12,0.083333,272.0,1776,0.153153
61202,12,2090.0,0.0,5,5664,3,5,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,12,0.083333,36.0,336,0.107143
61203,12,2599.0,6.0,5,3734,23,5,16.0,20.0,0.0,...,46.0,8.0,10.0,0.0,67.0,12,5.583333,272.0,1777,0.153067
61204,12,599.0,0.0,5,6710,30,5,0.0,0.0,0.0,...,97.0,85.0,57.0,0.0,0.0,12,0.000000,1051.0,1632,0.643995


In [21]:
feature_df.to_hdf(DATA + 'features.h5', key='features', format='table', mode='w')

In [128]:
%%time
tmp = pd.read_hdf(DATA + 'features.h5', key='features')
tmp.head()

Wall time: 23.9 s


In [129]:
%%time
tmp = pd.read_csv(DATA + 'features.csv')
tmp.head()

Wall time: 30.8 s


In [2]:
feature_df = pd.read_hdf(DATA + 'features.h5', key='features')

In [8]:
feature_df.shape

feature_df.head()

(4712400, 40)

,date_block_num,item_price,item_cnt_day,shop_id,item_id,item_category_id,city_id,item_lag_1,item_lag_3,item_lag_6,...,item_category_city_lag_6,item_category_city_lag_9,item_category_city_lag_12,item_city_cumsum,item_city_cumcount,item_city_avg,item_category_city_cumsum,item_category_city_cumcount,item_category_city_avg,month
61200,12,6999.0,0.0,5,3839,24,5,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,12,0.000000,0.0,1140,0.000000,1
61201,12,899.0,0.0,5,6415,23,5,0.0,1.0,0.0,...,8.0,10.0,0.0,1.0,12,0.083333,272.0,1776,0.153153,1
61202,12,2090.0,0.0,5,5664,3,5,1.0,0.0,0.0,...,0.0,0.0,0.0,1.0,12,0.083333,36.0,336,0.107143,1
61203,12,2599.0,6.0,5,3734,23,5,16.0,20.0,0.0,...,8.0,10.0,0.0,67.0,12,5.583333,272.0,1777,0.153067,1
61204,12,599.0,0.0,5,6710,30,5,0.0,0.0,0.0,...,85.0,57.0,0.0,0.0,12,0.000000,1051.0,1632,0.643995,1


In [4]:
feature_df[feature_df.date_block_num == 12].shape

(214200, 39)

In [7]:
feature_df['month'] = (feature_df.date_block_num % 12) + 1

In [19]:
days_per_month = pd.Series([31, 28, 31, 30, 31, 30, 31, 31, 30, 31, 30, 31], index=range(1, 13), name='days')

In [20]:
days_per_month.head()

1    31
2    28
3    31
4    30
5    31
Name: days, dtype: int64

In [22]:
feature_df = feature_df.merge(days_per_month.to_frame(), how="left", left_on='month', right_index=True)

In [23]:
feature_df.head()

,date_block_num,item_price,item_cnt_day,shop_id,item_id,item_category_id,city_id,item_lag_1,item_lag_3,item_lag_6,...,item_category_city_lag_9,item_category_city_lag_12,item_city_cumsum,item_city_cumcount,item_city_avg,item_category_city_cumsum,item_category_city_cumcount,item_category_city_avg,month,days
61200,12,6999.0,0.0,5,3839,24,5,0.0,0.0,0.0,...,0.0,0.0,0.0,12,0.000000,0.0,1140,0.000000,1,31
61201,12,899.0,0.0,5,6415,23,5,0.0,1.0,0.0,...,10.0,0.0,1.0,12,0.083333,272.0,1776,0.153153,1,31
61202,12,2090.0,0.0,5,5664,3,5,1.0,0.0,0.0,...,0.0,0.0,1.0,12,0.083333,36.0,336,0.107143,1,31
61203,12,2599.0,6.0,5,3734,23,5,16.0,20.0,0.0,...,10.0,0.0,67.0,12,5.583333,272.0,1777,0.153067,1,31
61204,12,599.0,0.0,5,6710,30,5,0.0,0.0,0.0,...,57.0,0.0,0.0,12,0.000000,1051.0,1632,0.643995,1,31


In [24]:
feature_df.columns

Index(['date_block_num', 'item_price', 'item_cnt_day', 'shop_id', 'item_id',
       'item_category_id', 'city_id', 'item_lag_1', 'item_lag_3', 'item_lag_6',
       'item_lag_9', 'item_lag_12', 'item_category_lag_1',
       'item_category_lag_3', 'item_category_lag_6', 'item_category_lag_9',
       'item_category_lag_12', 'item_cumsum', 'item_cumcount', 'item_avg',
       'item_category_cumsum', 'item_category_cumcount', 'item_category_avg',
       'item_city_lag_1', 'item_city_lag_3', 'item_city_lag_6',
       'item_city_lag_9', 'item_city_lag_12', 'item_category_city_lag_1',
       'item_category_city_lag_3', 'item_category_city_lag_6',
       'item_category_city_lag_9', 'item_category_city_lag_12',
       'item_city_cumsum', 'item_city_cumcount', 'item_city_avg',
       'item_category_city_cumsum', 'item_category_city_cumcount',
       'item_category_city_avg', 'month', 'days'],
      dtype='object')

In [25]:
templates = ['item_lag_%s', 'item_category_lag_%s', 'item_city_lag_%s', 'item_category_city_lag_%s']
columns = [t % i for t in templates for i in TIME_LAG]

In [30]:
feature_df.drop([c for c in feature_df.columns if c.endswith('_cumsum') or c.endswith('_cumcount')], inplace=True, axis=1)

In [33]:
feature_df.shape
feature_df.head()

(4712400, 33)

,date_block_num,item_price,item_cnt_day,shop_id,item_id,item_category_id,city_id,item_lag_1,item_lag_3,item_lag_6,...,item_city_lag_12,item_category_city_lag_1,item_category_city_lag_3,item_category_city_lag_6,item_category_city_lag_9,item_category_city_lag_12,item_city_avg,item_category_city_avg,month,days
61200,12,6999.0,0.0,5,3839,24,5,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,1,31
61201,12,899.0,0.0,5,6415,23,5,0.0,1.0,0.0,...,0.0,57.0,46.0,8.0,10.0,0.0,0.083333,0.153153,1,31
61202,12,2090.0,0.0,5,5664,3,5,1.0,0.0,0.0,...,0.0,31.0,0.0,0.0,0.0,0.0,0.083333,0.107143,1,31
61203,12,2599.0,6.0,5,3734,23,5,16.0,20.0,0.0,...,0.0,57.0,46.0,8.0,10.0,0.0,5.583333,0.153067,1,31
61204,12,599.0,0.0,5,6710,30,5,0.0,0.0,0.0,...,0.0,149.0,97.0,85.0,57.0,0.0,0.000000,0.643995,1,31


In [38]:
for c in tqdm_notebook(columns):
    feature_df[c] = feature_df[c] / feature_df.days

In [40]:
feature_df.shape

feature_df.head()

(4712400, 33)

,date_block_num,item_price,item_cnt_day,shop_id,item_id,item_category_id,city_id,item_lag_1,item_lag_3,item_lag_6,...,item_city_lag_12,item_category_city_lag_1,item_category_city_lag_3,item_category_city_lag_6,item_category_city_lag_9,item_category_city_lag_12,item_city_avg,item_category_city_avg,month,days
61200,12,6999.0,0.0,5,3839,24,5,0.000000,0.000000,0.0,...,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,1,31
61201,12,899.0,0.0,5,6415,23,5,0.000000,0.032258,0.0,...,0.0,1.838710,1.483871,0.258065,0.322581,0.0,0.083333,0.153153,1,31
61202,12,2090.0,0.0,5,5664,3,5,0.032258,0.000000,0.0,...,0.0,1.000000,0.000000,0.000000,0.000000,0.0,0.083333,0.107143,1,31
61203,12,2599.0,6.0,5,3734,23,5,0.516129,0.645161,0.0,...,0.0,1.838710,1.483871,0.258065,0.322581,0.0,5.583333,0.153067,1,31
61204,12,599.0,0.0,5,6710,30,5,0.000000,0.000000,0.0,...,0.0,4.806452,3.129032,2.741935,1.838710,0.0,0.000000,0.643995,1,31


In [41]:
feature_df.to_hdf(DATA + 'features_v1.h5', key='features', format='table', mode='w')